In [167]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely import wkt

ROOT = Path('../..')
ROOT.resolve()

PosixPath('/Users/lukestrange/Code/housing')

In [168]:
brownfield_sites = pd.read_csv(ROOT / "raw/brownfield/brownfield-land.csv")
lad_geojson = pd.read_json(ROOT / "src/_data/geojson/test.geojson", orient='columns')

In [169]:
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None

In [170]:
# Load the points dataframe (assuming it's a pandas DataFrame)
brownfield_sites['geometry'] = brownfield_sites['point'].apply(wkt_loads)
points_gdf = gpd.GeoDataFrame(brownfield_sites, geometry='geometry')

# Load the polygons GeoJSON
polygons_gdf = gpd.read_file(ROOT / 'src/_data/geojson/Local_Authority_Districts_May_2024_Boundaries_UK_BGC_Simplified.json')

# Ensure the same CRS
points_gdf = points_gdf.set_crs(polygons_gdf.crs, allow_override=True)

# Perform spatial join
joined_gdf = gpd.sjoin(points_gdf, polygons_gdf, how="left", predicate="within")

# Now, joined_gdf will have a 'name' column from the polygons
name_counts = joined_gdf[['LAD24NM', 'LAD24CD']].value_counts()
name_counts.name = 'sites'

In [171]:
joined_gdf = joined_gdf[['LAD24CD', 'LAD24NM', 'hectares', 'maximum-net-dwellings', 'minimum-net-dwellings']].groupby(['LAD24CD', 'LAD24NM']).sum(numeric_only=True).reset_index()
joined_gdf = joined_gdf.join(name_counts, on=['LAD24NM', 'LAD24CD'], how='inner')
joined_gdf.to_csv(ROOT / 'data/brownfield/sites_by_local_authority.csv', index=False)